# Chapter7 RNN을 사용한 문장 생성

## 7.1 언어 모델을 사용한 문장 생성

### 7.1.1 RNN을 사용한 문장 생성의 순서
- 확률이 가장 높은 단어를 선택 반복

### 7.1.2 문장 생성 구현

In [1]:
import sys
sys.path.append('..')
import numpy as np
from common.functions import softmax
from common.rnnlm import Rnnlm
from common.better_rnnlm import BetterRnnlm


class RnnlmGen(Rnnlm):
    def generate(self, start_id, skip_ids=None, sample_size=100):
        word_ids = [start_id]

        x = start_id
        while len(word_ids) < sample_size:
            x = np.array(x).reshape(1, 1)
            score = self.predict(x)
            p = softmax(score.flatten())

            sampled = np.random.choice(len(p), size=1, p=p)
            if (skip_ids is None) or (sampled not in skip_ids):
                x = sampled
                word_ids.append(int(x))

        return word_ids

    def get_state(self):
        return self.lstm_layer.h, self.lstm_layer.c

    def set_state(self, state):
        self.lstm_layer.set_state(*state)

### 7.1.3 더 좋은 문장으로

## 7.2 seq2seq

### 7.2.1 seq2seq의 원리

### 7.2.2 시계열 데이터 변환용 장난감 문제

### 7.2.3 가변 길이 시계열 데이터

### 7.2.4 덧셈 데이터셋

## 7.3 seq2seq 구현

### 7.3.1 Encoder 클래스

### 7.3.2 Decoder 클래스

### 7.3.3 seq2seq 클래스

### 7.3.4 seq2seq 평가

## 7.4 seq2seq 개선

### 7.4.1 입력 데이터 반전 (Reverse)

### 7.4.2 엿보기 (Peeky)

## 7.5 seq2seq를 이용하는 애플리케이션

### 7.5.1 챗봇

### 7.5.2 알고리즘 학습

### 7.5.3 이미지 캡셔닝